In [1]:
import json
import jsonlines
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
from torchvision import transforms

source_dir = "/scratch/lt2326-h21/a1/"
image_dir = "/scratch/lt2326-h21/a1/images/"
info_filename = "/scratch/lt2326-h21/a1/info.json"

In [2]:
image_files = [f for f in listdir(image_dir) if isfile(join(image_dir, f))]

In [3]:
with open(info_filename, 'r') as info_file:
    data = info_file.read()
info_obj = json.loads(data)

In [4]:
df_train = pd.DataFrame.from_dict(info_obj['train'])
df_val = pd.DataFrame.from_dict(info_obj['val'])
df = pd.concat([df_train, df_val])

In [5]:
df = df[pd.DataFrame(df.file_name.to_list()).isin(image_files).any(1).values]

In [6]:
bb_data = []

with jsonlines.open(source_dir + "train.jsonl") as reader:
   for line in reader.iter(type=dict):
      if (line.get('file_name') in image_files):
         bb_data.append(line)

with jsonlines.open(source_dir + "val.jsonl") as reader:
   for line in reader.iter(type=dict):
      if (line.get('file_name') in image_files):
         bb_data.append(line)

In [ ]:
bb_data

In [4]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
testimg = mpimg.imread(image_dir + "0000486.jpg")
imgarray= np.array(testimg)
convert_tensor = transforms.ToTensor()

/usr/local/lib64/python3.9/site-packages/torchvision/transforms/functional.py:114: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)


In [3]:
convert_tensor(imgarray)

tensor([[[0.5137, 0.5373, 0.4863,  ..., 0.6863, 0.6863, 0.6863],
         [0.4863, 0.5490, 0.4941,  ..., 0.6863, 0.6863, 0.6863],
         [0.4510, 0.5529, 0.4941,  ..., 0.6863, 0.6863, 0.6863],
         ...,
         [0.4000, 0.4078, 0.4196,  ..., 0.5176, 0.5020, 0.4784],
         [0.4078, 0.4314, 0.4510,  ..., 0.5137, 0.5216, 0.5137],
         [0.4118, 0.4314, 0.4353,  ..., 0.4902, 0.5294, 0.5216]],

        [[0.5608, 0.5922, 0.5490,  ..., 0.7725, 0.7725, 0.7725],
         [0.5333, 0.6039, 0.5569,  ..., 0.7725, 0.7725, 0.7725],
         [0.4863, 0.5961, 0.5490,  ..., 0.7725, 0.7725, 0.7725],
         ...,
         [0.3922, 0.4000, 0.4118,  ..., 0.5098, 0.4941, 0.4706],
         [0.4000, 0.4235, 0.4431,  ..., 0.5059, 0.5137, 0.5059],
         [0.4039, 0.4235, 0.4275,  ..., 0.4824, 0.5216, 0.5137]],

        [[0.6549, 0.6941, 0.6471,  ..., 0.9333, 0.9255, 0.9255],
         [0.6353, 0.7059, 0.6549,  ..., 0.9333, 0.9255, 0.9255],
         [0.6000, 0.7059, 0.6549,  ..., 0.9333, 0.9255, 0.